- **Module:** read_map_aeronet_inv.ipynb
- **Authors:** Petar Grigorov and Pawan Gupta
- **Organization:** NASA AERONET (https://aeronet.gsfc.nasa.gov/)
- **Date:** 06/18/2023
- **Last Revision:** 07/29/2024
- **Purpose:** To access and map AERONET Inversion products from Web API
- **Disclaimer:** The code is for demonstration purposes only. Users are responsible to check for accuracy and revise to fit their objective.
- **Contact:** Report any concern or question related to the code to pawan.gupta@nasa.gov or petar.t.grigorov@nasa.gov
- **Readme:** https://github.com/pawanpgupta/AERONET/blob/Python/README/Read_and_map_AERONET

**Required packages installation and importing**

In [ ]:
!pip uninstall -y numpy pandas shapely
!pip install numpy==1.26.4 pandas==2.0.3 shapely==1.8.5
!pip install cartopy
!pip install beautifulsoup4
!pip install requests
!pip install geopandas

from bs4 import BeautifulSoup      #reads data from website (web scraping)
import re                          #regular expression matching operations (RegEx)
import os                          #for working with directories
import requests                    #useful for sending HTTP requests using python
import shutil                      #useful for creating zip files
import numpy as np                 #for array manipulation
import pandas as pd                #for data querying and processing
import geopandas as gpd            #same as pandas, but for geospatial data
import matplotlib.pyplot as plt    #for creating plots

import cartopy.crs as ccrs         #for creating geographical maps
import cartopy.feature as cfeature
from copy import deepcopy as dc

import warnings
warnings.filterwarnings('ignore')

**Outputs**

In [ ]:
from google.colab import files      #ensures output zip file can be downloaded
from google.colab import drive      #imports local google drive
drive.mount('/drive')               #mounts local google drive onto colab
!mkdir Output                       #makes directory where output files will be stored

**Inputs**

In [ ]:
dt_initial = '20240627'  #Must be in YYYYMMDD format
dt_final = '20240629'
level = 1.5              #Level 1.0, 1.5, or 2.0 available
avg = 2                  #Daily averages (1), Hourly (2), or site total (3)
feature_choice = 'SSA'   #Available Inversion products: CAD, TAB, AOD, SSA
inv_type = 'ALM'         #Available Inversion types: ALM, HYB
wave = 440               #Wavelength of interest
#Bounding box: Coordinates must be in decimal degrees (including decimal)
lat1,lon1 = 23.,-135.    #lat1,lon1 - Lower Left
lat2,lon2 = 53.,-65.     #lat2,lon2 - Upper Right

**Validation**

In [ ]:
dt_initial = dt_initial.replace("-","").replace("/","").replace(" ","")
dt_final = dt_final.replace("-","").replace("/","").replace(" ","")

level = str(level).replace(".","")
feature_choice = feature_choice.upper()
inv_type = inv_type.upper()

if avg == 1:
    average_type = '20'
elif avg == 2 or avg == 3:
    average_type = '10'
else:
    average_type = '20' #will default to daily averages for improper input

if feature_choice == 'SSA':
    vis_min = 0.90
    vis_max = 1.00
elif feature_choice == 'CAD':
    vis_min = 0.00
    vis_max = 0.25
elif feature_choice == 'TAB':
    vis_min = 0.00
    vis_max = 0.05
else:
    vis_min = 0.00
    vis_max = 0.25

**Processing**

In [ ]:
yr_initial = dt_initial[:4]               #initial year
mon_initial = dt_initial[4:6]             #initial month
day_initial = dt_initial[6:]              #initial day

yr_final = dt_final[:4]                   #final year
mon_final = dt_final[4:6]                 #final month
day_final = dt_final[6:]                  #final day

url = 'https://aeronet.gsfc.nasa.gov/cgi-bin/print_web_data_inv_v3?lat1='+str(lat1)+'&lon1='+str(lon1)+'&lat2='+str(lat2)+'&lon2='+str(lon2)+'&year='+yr_initial+'&month='+mon_initial+'&day='+day_initial+'&year2='+yr_final+'&month2='+mon_final+'&day2='+day_final+'&product='+feature_choice+'&AVG='+average_type+'&'+inv_type+str(level)+'=1'
soup = BeautifulSoup(requests.get(url).text) #web services contents are read here from URL

**Main**

In [ ]:
if len(soup.find_all('br')) > 0:

    indir = r'/content/sample_data/soup.txt'
    with open(indir ,"w") as oFile:          #writes the data scraped from "beautiful soup" to a text file on your local Google drive
        oFile.write(str(soup.text))
        oFile.close()

    df = pd.read_csv(indir,skiprows = 7)     #loads the csv data into a Pandas dataframe
    os.remove(indir)
    df = df.replace(-999.0, np.nan)                                     #replaces all -999.0 vakyes with NaN; helps with accurate data aggregation
    df[['Day','Month','Year']] = df['Date(dd:mm:yyyy)'].str.split(':',expand=True)                                #splits the date column and then joins it back together using "-" instead of ":"
    df['Date'] = df[['Year','Month','Day']].apply(lambda x: '-'.join(x.values.astype(str)), axis="columns")       #because datetime format in python does not recognize colons
    df['Hour'] = df['Time(hh:mm:ss)'].str[:2]                           #creates Hour column using just the HH component of the Time column
    df['Date']= pd.to_datetime(df['Date'])                                              #converts the new date column to datetime format

    df_nonwave = df[['Date','Latitude(Degrees)','Longitude(Degrees)']]
    df_wave = df.filter(like=str(wave)+'nm')

    if avg == 2:
        numeric_cols = df.select_dtypes(include='number').columns.tolist()
        df = df.groupby(['AERONET_Site','Date','Hour'])[numeric_cols].mean()
        df = df.reset_index()
        df_nonwave = df[['Date','Hour','Latitude(Degrees)','Longitude(Degrees)']]
        df_wave = df.filter(like=str(wave)+'nm')
    elif avg == 3:
        numeric_cols = df.select_dtypes(include='number').columns.tolist()
        df = df.groupby(['AERONET_Site'])[numeric_cols].mean()
        df = df.reset_index()
        df_nonwave = df[['Latitude(Degrees)','Longitude(Degrees)']]
        df_wave = df.filter(like=str(wave)+'nm')

    if not df_wave.empty:
        df = pd.concat([df_nonwave, df.filter(like=str(wave)+'nm')], axis=1)

        geo_df = dc(df)
        projection=ccrs.PlateCarree()
        os.makedirs('Output/PlateCarree', exist_ok=True)
        outdir_Plate='/content/Output/PlateCarree/'

        colbar='RdYlGn_r' #https://matplotlib.org/stable/tutorials/colors/colormaps.html

        if avg == 2:
            geo_df = dc(df)
            geo_df['Hour'] = pd.to_datetime(geo_df['Hour'].astype(str), format='%H')
            geo_df['Hour'] = geo_df['Hour'].dt.time
            geo_df['Date_Time'] = pd.to_datetime(geo_df['Date'].astype(str) + ' ' + geo_df['Hour'].astype(str))
            geo_df = geo_df.drop(columns=['Date', 'Hour'])

            date_list = geo_df[['Date_Time']].astype(str)
            date_list = date_list.to_numpy()
            date_list = np.unique(date_list)

            for i in range(len(date_list)):
                geo_df = dc(df)
                geo_df['Hour'] = pd.to_datetime(geo_df['Hour'].astype(str), format='%H')
                geo_df['Hour'] = geo_df['Hour'].dt.time
                geo_df['Date_Time'] = pd.to_datetime(geo_df['Date'].astype(str) + ' ' + geo_df['Hour'].astype(str))
                geo_df = geo_df.drop(columns=['Date', 'Hour'])

                fig, ax = plt.subplots(figsize=(16,12),subplot_kw={'projection': projection},frameon=True)
                plt.xlim([lon1,lon2])
                plt.ylim([lat1,lat2])

                # Comment next 2 lines if you don't want grey color map
                countries = gpd.read_file(gpd.datasets.get_path("naturalearth_lowres"))
                countries.plot(color="lightgrey",ax=ax,zorder=0,alpha=0.1)

                geo_df = geo_df.loc[geo_df[geo_df.columns[-1]] == date_list[i]].reset_index(drop=True)
                colbar = plt.cm.get_cmap('RdYlGn_r')
                colbar.set_extremes(under='darkgreen',over='magenta')
                cm = ax.scatter(x=geo_df[geo_df.columns[1]], y=geo_df[geo_df.columns[0]],
                                c=geo_df[geo_df.columns[2]],
                            cmap=colbar, vmin = vis_min, vmax = vis_max, s = 400,zorder=1)

                ax.set_title(date_list[i]+str(' GMT'),size=20, weight='bold')

                # Add coastlines
                ax.coastlines(resolution='10m',zorder=0)

                # Add state and country boundaries
                ax.add_feature(cfeature.STATES.with_scale('10m'), linewidth=0.5, edgecolor='lightgray',zorder=0)
                ax.add_feature(cfeature.BORDERS.with_scale('10m'), linewidth=0.5, edgecolor='lightgray',zorder=0)

                #Add colorbar
                cax = fig.add_axes([ax.get_position().x1+0.01,ax.get_position().y0,0.02,ax.get_position().height])

                cbar=plt.colorbar(cm, cax=cax, extend = 'both')
                cax.set_ylabel(geo_df.columns[2],size=20, weight='bold')
                cbar.ax.tick_params(labelsize=20)

                fig.savefig(outdir_Plate+str(date_list[i][:-6])+'_level'+str(level)+'_'+str(feature_choice)+'_'+geo_df.columns[2]+'_'+str(inv_type)+'_'+str(lon1)+'_'+str(lon2)+'_'+str(lat1)+'_'+str(lat2)+'_PlateCarree.png', bbox_inches='tight',dpi=400)

        elif avg == 3:
            geo_df = dc(df)
            fig, ax = plt.subplots(figsize=(16,12),subplot_kw={'projection': projection},frameon=True)

            # Comment next 2 lines if you don't want grey color map
            countries = gpd.read_file(gpd.datasets.get_path("naturalearth_lowres"))
            countries.plot(color="lightgrey",ax=ax,zorder=0,alpha=0.1)

            # Add coastlines
            ax.coastlines(resolution='10m',zorder=1)

            # Add state and country boundaries
            ax.add_feature(cfeature.STATES.with_scale('10m'), linewidth=0.5, edgecolor='lightgray',zorder=1)
            ax.add_feature(cfeature.BORDERS.with_scale('10m'), linewidth=0.5, edgecolor='lightgray',zorder=1)

            plt.xlim([lon1,lon2])
            plt.ylim([lat1,lat2])
            colbar = plt.cm.get_cmap('RdYlGn_r')
            colbar.set_extremes(under='darkgreen',over='magenta')
            cm = ax.scatter(x=geo_df[geo_df.columns[1]], y=geo_df[geo_df.columns[0]],
                              c=geo_df[geo_df.columns[2]],
                          cmap=colbar, vmin = vis_min, vmax = vis_max, s = 400,zorder=2)
            ax.set_title("Site Average",size=20, weight='bold')

            #Add colorbar
            cax = fig.add_axes([ax.get_position().x1+0.01,ax.get_position().y0,0.02,ax.get_position().height])
            cbar=plt.colorbar(cm, cax=cax, extend = 'both')
            cax.set_ylabel(geo_df.columns[2],size=20, weight='bold')
            cbar.ax.tick_params(labelsize=20)

            fig.savefig(outdir_Plate+'SiteAverage_level'+str(level)+'_'+str(feature_choice)+'_'+geo_df.columns[2]+'_'+str(inv_type)+'_'+str(lon1)+'_'+str(lon2)+'_'+str(lat1)+'_'+str(lat2)+'_PlateCarree.png',bbox_inches='tight',dpi=400)

        else:
            #for daily averages
            date_list = geo_df[['Date']].astype(str)
            date_list = date_list.to_numpy()
            date_list = np.unique(date_list)

            for i in range(len(date_list)):
                geo_df = dc(df)

                fig, ax = plt.subplots(figsize=(16,12),subplot_kw={'projection': projection},frameon=True)
                plt.xlim([lon1,lon2])
                plt.ylim([lat1,lat2])

                # Comment next 2 lines if you don't want grey color map
                countries = gpd.read_file(gpd.datasets.get_path("naturalearth_lowres"))
                countries.plot(color="lightgrey",ax=ax,zorder=0,alpha=0.1)

                geo_df = geo_df.loc[geo_df['Date'] == date_list[i]].reset_index(drop=True)
                colbar = plt.cm.get_cmap('RdYlGn_r')
                colbar.set_extremes(under='darkgreen',over='magenta')
                cm = ax.scatter(x=geo_df[geo_df.columns[2]], y=geo_df[geo_df.columns[1]],
                            c=geo_df[geo_df.columns[3]],
                            cmap=colbar, vmin = vis_min, vmax = vis_max, s = 400,zorder=1)
                ax.set_title(date_list[i],size=20, weight='bold')

                # Add coastlines
                ax.coastlines(resolution='10m',zorder=0)

                # Add state and country boundaries
                ax.add_feature(cfeature.STATES.with_scale('10m'), linewidth=0.5, edgecolor='lightgray',zorder=0)
                ax.add_feature(cfeature.BORDERS.with_scale('10m'), linewidth=0.5, edgecolor='lightgray',zorder=0)

                #Add colorbar
                cax = fig.add_axes([ax.get_position().x1+0.01,ax.get_position().y0,0.02,ax.get_position().height])
                cbar=plt.colorbar(cm, cax=cax, extend = 'both')
                cax.set_ylabel(geo_df.columns[3],size=20, weight='bold')
                cbar.ax.tick_params(labelsize=20)

                fig.savefig(outdir_Plate+str(date_list[i])+'_level'+str(level)+'_'+str(feature_choice)+'_'+geo_df.columns[3]+'_'+str(inv_type)+'_'+str(lon1)+'_'+str(lon2)+'_'+str(lat1)+'_'+str(lat2)+'.png',bbox_inches='tight',dpi=400)

        ## Define central location for Orthogrphic projection
        central_longitude = lon1 + abs(lon2 - lon2)//2
        central_latitude  = lat1 + abs(lat1 - lat2)//2

        ### ORTHO PROJECTION CELL
        projection=ccrs.Orthographic(central_longitude=central_longitude, central_latitude=central_latitude)
        geo = ccrs.Geodetic()

        os.makedirs('Output/Orthographic', exist_ok=True)
        outdir_Ortho ='/content/Output/Orthographic/'

        colbar='RdYlGn_r' #https://matplotlib.org/stable/tutorials/colors/colormaps.html

        if avg == 2:
            geo_df = dc(df)
            geo_df['Hour'] = pd.to_datetime(geo_df['Hour'].astype(str), format='%H')
            geo_df['Hour'] = geo_df['Hour'].dt.time
            geo_df['Date_Time'] = pd.to_datetime(geo_df['Date'].astype(str) + ' ' + geo_df['Hour'].astype(str))
            geo_df = geo_df.drop(columns=['Date', 'Hour'])

            date_list = geo_df[['Date_Time']].astype(str)
            date_list = date_list.to_numpy()
            date_list = np.unique(date_list)

            for i in range(len(date_list)):
                geo_df = dc(df)
                geo_df['Hour'] = pd.to_datetime(geo_df['Hour'].astype(str), format='%H')
                geo_df['Hour'] = geo_df['Hour'].dt.time
                geo_df['Date_Time'] = pd.to_datetime(geo_df['Date'].astype(str) + ' ' + geo_df['Hour'].astype(str))
                geo_df = geo_df.drop(columns=['Date', 'Hour'])

                fig, ax = plt.subplots(figsize=(16,12),subplot_kw={'projection': projection},frameon=True)
                ax.set_extent([lon1,lon2,lat1,lat2],crs=ccrs.PlateCarree())

                # Comment next 2 lines if you don't want grey color map
                countries = gpd.read_file(gpd.datasets.get_path("naturalearth_lowres"))
                countries.plot(color="lightgrey",ax=ax,zorder=0,alpha=0.1,transform=ccrs.PlateCarree())

                geo_df = geo_df.loc[geo_df[geo_df.columns[-1]] == date_list[i]].reset_index(drop=True)
                colbar = plt.cm.get_cmap('RdYlGn_r')
                colbar.set_extremes(under='darkgreen',over='magenta')
                cm = ax.scatter(x=geo_df[geo_df.columns[1]], y=geo_df[geo_df.columns[0]],
                                c=geo_df[geo_df.columns[2]],
                            cmap=colbar, vmin = vis_min, vmax = vis_max, s = 400, zorder=1, transform=ccrs.PlateCarree())
                ax.set_title(date_list[i]+str(' GMT'),size=20, weight='bold')

                # Add coastlines
                ax.coastlines(resolution='10m',zorder=0)

                # Add state and country boundaries
                ax.add_feature(cfeature.STATES.with_scale('10m'), linewidth=0.5, edgecolor='lightgray',zorder=0)
                ax.add_feature(cfeature.BORDERS.with_scale('10m'), linewidth=0.5, edgecolor='lightgray',zorder=0)

                #Add colorbar
                cax = fig.add_axes([ax.get_position().x1+0.01,ax.get_position().y0,0.02,ax.get_position().height])
                cbar=plt.colorbar(cm, cax=cax, extend='both')
                cax.set_ylabel(geo_df.columns[2],size=20, weight='bold')
                cbar.ax.tick_params(labelsize=20)

                fig.savefig(outdir_Ortho+str(date_list[i][:-6])+'_level'+str(level)+'_'+str(feature_choice)+'_'+geo_df.columns[2]+'_'+str(inv_type)+'_'+str(lon1)+'_'+str(lon2)+'_'+str(lat1)+'_'+str(lat2)+'.png')

        elif avg == 3:
            geo_df = dc(df)
            fig, ax = plt.subplots(figsize=(16,12),subplot_kw={'projection': projection},frameon=True)
            ax.set_extent([lon1,lon2,lat1,lat2],crs=ccrs.PlateCarree())

            # Comment next 2 lines if you don't want grey color map
            countries = gpd.read_file(gpd.datasets.get_path("naturalearth_lowres"))
            countries.plot(color="lightgrey",ax=ax,zorder=0,alpha=0.1)

            # Add coastlines
            ax.coastlines(resolution='10m',zorder=1)

            # Add state and country boundaries
            ax.add_feature(cfeature.STATES.with_scale('10m'), linewidth=0.5, edgecolor='lightgray',zorder=1)
            ax.add_feature(cfeature.BORDERS.with_scale('10m'), linewidth=0.5, edgecolor='lightgray',zorder=1)

            colbar = plt.cm.get_cmap('RdYlGn_r')
            colbar.set_extremes(under='darkgreen',over='magenta')
            cm = ax.scatter(x=geo_df[geo_df.columns[1]], y=geo_df[geo_df.columns[0]],
                              c=geo_df[geo_df.columns[2]],
                          cmap=colbar, vmin = vis_min, vmax = vis_max, s = 400,zorder=2,transform=ccrs.PlateCarree())

            ax.set_title("Site Averages",size=20, weight='bold')

            #Add colorbar
            cax = fig.add_axes([ax.get_position().x1+0.01,ax.get_position().y0,0.02,ax.get_position().height])
            plt.colorbar(cm, cax=cax, extend='both')
            cax.set_ylabel(geo_df.columns[2],size=20, weight='bold')

            fig.savefig(outdir_Ortho+'SiteAverage_level'+str(level)+'_'+str(feature_choice)+'_'+geo_df.columns[2]+'_'+str(inv_type)+'_'+str(lon1)+'_'+str(lon2)+'_'+str(lat1)+'_'+str(lat2)+'.png')

        else:
            #for daily averages
            geo_df = dc(df)
            date_list = geo_df[['Date']].astype(str)
            date_list = date_list.to_numpy()
            date_list = np.unique(date_list)

            for i in range(len(date_list)):
                geo_df = dc(df)

                fig, ax = plt.subplots(figsize=(16,12),subplot_kw={'projection': projection},frameon=True)
                ax.set_extent([lon1,lon2,lat1,lat2],crs=ccrs.PlateCarree())

                # Comment next 2 lines if you don't want grey color map
                countries = gpd.read_file(gpd.datasets.get_path("naturalearth_lowres"))
                countries.plot(color="lightgrey",ax=ax,zorder=0,alpha=0.1)

                geo_df = geo_df.loc[geo_df['Date'] == date_list[i]].reset_index(drop=True)
                colbar = plt.cm.get_cmap('RdYlGn_r')
                colbar.set_extremes(under='darkgreen',over='magenta')
                cm = ax.scatter(x=geo_df[geo_df.columns[2]], y=geo_df[geo_df.columns[1]],
                            c=geo_df[geo_df.columns[3]],
                            cmap=colbar, vmin = vis_min, vmax = vis_max, s = 400,zorder=1,transform=ccrs.PlateCarree())

                ax.set_title(date_list[i],size=20, weight='bold')

                # Add coastlines
                ax.coastlines(resolution='10m',zorder=0)

                # Add state and country boundaries
                ax.add_feature(cfeature.STATES.with_scale('10m'), linewidth=0.5, edgecolor='lightgray',zorder=0)
                ax.add_feature(cfeature.BORDERS.with_scale('10m'), linewidth=0.5, edgecolor='lightgray',zorder=0)

                #Add colorbar
                cax = fig.add_axes([ax.get_position().x1+0.01,ax.get_position().y0,0.02,ax.get_position().height])
                cbar=plt.colorbar(cm, cax=cax, extend='both')
                cax.set_ylabel(geo_df.columns[3],size=20, weight='bold')
                cbar.ax.tick_params(labelsize=20)

                fig.savefig(outdir_Ortho+str(date_list[i])+'_level'+str(level)+'_'+str(feature_choice)+'_'+geo_df.columns[3]+'_'+str(inv_type)+'_'+str(lon1)+'_'+str(lon2)+'_'+str(lat1)+'_'+str(lat2)+'.png')

    else:
        print("\nWavelength not found. Please try again.")

else:
    print("\nNo data to read. Please try again.")

**Zip file download**

In [ ]:
while True:
  zip_download = str(input("Would you like to download your output in a zipped folder (y or n)?: "))
  if zip_download == 'y' or zip_download == 'Y' or zip_download == 'Yes' or zip_download == 'yes':
    shutil.make_archive('Output', 'zip', '/content/Output')  #zips all output files
    files.download('Output.zip')  #Note: Must use Chrome browser for download to work
    break
  elif zip_download == 'n' or zip_download == 'N' or zip_download == 'No' or zip_download == 'no':
    print("\nThanks! I hope you enjoyed the program.")
    break
  else:
    print("\nIncorrect input. Please try again!")